In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

import datetime


from pyScript import get_tweets, cleanup_columns, cleanup_rows, display_img, top_characteristics

%matplotlib inline

# Web/Twitter scraping
Here I download the first n number of twitts about COVID from the first to the last days of January 2020.

In [2]:
# get_tweets('(from:jimmyfallon) until:2009-01-01 since:2008-12-01', 'data_08')

In [3]:
# get_tweets('(from:jimmyfallon) until:2019-01-01 since:2018-12-01', 'data_18')

In [4]:
data_08 = pd.read_csv('data_08.csv')
data_18 = pd.read_csv('data_18.csv')

In [5]:
data_08.head()

,Unnamed: 0,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,...,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
0,0,jimmyfallon,jimmy fallon,15485441,1088339520,/jimmyfallon/status/1088339520,2008-12-31 17:09:25,1230743365,People on the streets already. Getting their s...,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,3,0,0,False,False,NaN,[]
1,1,jimmyfallon,jimmy fallon,15485441,1088336553,/jimmyfallon/status/1088336553,2008-12-31 17:07:50,1230743270,Times Square not shut down yet. 3pm it all clo...,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,2,0,0,False,False,NaN,[]
2,2,jimmyfallon,jimmy fallon,15485441,1087884053,/jimmyfallon/status/1087884053,2008-12-31 12:50:52,1230727852,Check out the new office in today's video. Als...,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,4,0,0,False,False,NaN,[]
3,3,jimmyfallon,jimmy fallon,15485441,1087176379,/jimmyfallon/status/1087176379,2008-12-31 02:45:12,1230691512,"Starting to countdown now.. 94,500... 94,499.....","<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,8,0,0,False,False,NaN,[]
4,4,jimmyfallon,jimmy fallon,15485441,1087055653,/jimmyfallon/status/1087055653,2008-12-31 01:30:14,1230687014,Ideal New Years Eve?,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,4,0,0,False,False,NaN,[]


In [6]:
list(data_08)

['Unnamed: 0',
 'screen_name',
 'username',
 'user_id',
 'tweet_id',
 'tweet_url',
 'timestamp',
 'timestamp_epochs',
 'text',
 'text_html',
 'links',
 'hashtags',
 'has_media',
 'img_urls',
 'video_url',
 'likes',
 'retweets',
 'replies',
 'is_replied',
 'is_reply_to',
 'parent_tweet_id',
 'reply_to_users']

In [7]:
_header = ['tweetId', 'tweetUrl', 'timestamp', 'text', 'hashtags', 'hasMedia', 'imgUrl', 'likes', 'retweets', 'replies', 'isReplied']
_drop = ['Unnamed: 0', 'screen_name', 'username', 'user_id', 'timestamp_epochs', 'text_html', 'links', 'video_url', 'is_reply_to', 'parent_tweet_id', 'reply_to_users']

In [8]:
data_08 = cleanup_columns(data_08, names=_header, to_drop=_drop)
data_08

,tweetId,tweetUrl,timestamp,text,hashtags,hasMedia,imgUrl,likes,retweets,replies,isReplied
0,1088339520,/jimmyfallon/status/1088339520,2008-12-31 17:09:25,People on the streets already. Getting their s...,[],False,[],3,0,0,False
1,1088336553,/jimmyfallon/status/1088336553,2008-12-31 17:07:50,Times Square not shut down yet. 3pm it all clo...,[],False,[],2,0,0,False
2,1087884053,/jimmyfallon/status/1087884053,2008-12-31 12:50:52,Check out the new office in today's video. Als...,[],False,[],4,0,0,False
3,1087176379,/jimmyfallon/status/1087176379,2008-12-31 02:45:12,"Starting to countdown now.. 94,500... 94,499.....",[],False,[],8,0,0,False
4,1087055653,/jimmyfallon/status/1087055653,2008-12-31 01:30:14,Ideal New Years Eve?,[],False,[],4,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...
116,1049293035,/jimmyfallon/status/1049293035,2008-12-10 15:21:30,@qoolquest Me twitter now.,[],False,[],5,1,0,False
117,1049262977,/jimmyfallon/status/1049262977,2008-12-10 15:05:32,hey. psyched that people liked the show yester...,[],False,[],8,6,0,False
118,1041155237,/jimmyfallon/status/1041155237,2008-12-05 23:38:21,is ready to launch the show on Monday!,[],False,[],8,3,0,False
119,1038690810,/jimmyfallon/status/1038690810,2008-12-04 19:15:08,is listening to music :-),[],False,[],90,42,6,True


In [9]:
data_18 = cleanup_columns(data_18, names=_header, to_drop=_drop)
data_18

,tweetId,tweetUrl,timestamp,text,hashtags,hasMedia,imgUrl,likes,retweets,replies,isReplied
0,1079510916318679040,/jimmyfallon/status/1079510916318679040,2018-12-30 22:53:53,Oooohhhhhhhh my man #JIMIN (@BTS_twt) with a s...,['JIMIN'],False,[],486327,161313,6928,True
1,1078658709721468929,/jimmyfallon/status/1078658709721468929,2018-12-28 14:27:31,Tonight Show Retro Game Challenge vs @Ninja. S...,[],False,[],3919,258,172,True
2,1076337366007246849,/jimmyfallon/status/1076337366007246849,2018-12-22 04:43:20,Watching the best of 2018 #FallonTonight now.,['FallonTonight'],False,[],5880,312,288,True
3,1076273833928212480,/jimmyfallon/status/1076273833928212480,2018-12-22 00:30:52,Haaaa!!! Thank you my brother. It’s delicious ...,['TonightDoughhttps'],False,[],2476,132,107,True
4,1076117140351172608,/jimmyfallon/status/1076117140351172608,2018-12-21 14:08:14,Tonight! We're looking back at our favorite mo...,['FallonTonight'],False,[],4264,314,162,True
...,...,...,...,...,...,...,...,...,...,...,...
61,1069989127892975617,/jimmyfallon/status/1069989127892975617,2018-12-04 16:17:42,It’s Hashtags time! Tweet out a funny or weird...,['DecorationDisaster'],False,[],1037,97,282,True
62,1069971768650199041,/jimmyfallon/status/1069971768650199041,2018-12-04 15:08:43,"TONIGHT! Saoirse Ronan, @russwest44, and a per...",['FallonTonight'],False,[],904,108,47,True
63,1069941448366321666,/jimmyfallon/status/1069941448366321666,2018-12-04 13:08:14,Loaded Questions with @MargotRobbie! https://y...,['FallonTonight'],False,[],663,74,41,True
64,1069617267732819968,/jimmyfallon/status/1069617267732819968,2018-12-03 15:40:03,"Fun show tonight: @MargotRobbie is here, @Elsi...",['FallonTonight'],False,[],1054,89,66,True


# Data Mining

In [10]:
data_08.isna().sum()

tweetId      0
tweetUrl     0
timestamp    0
text         0
hashtags     0
hasMedia     0
imgUrl       0
likes        0
retweets     0
replies      0
isReplied    0
dtype: int64

In [11]:
# dropping the duplicates and nans
print(f'Shape before: {data_08.shape[0]}')
data_08 = cleanup_rows(df=data_08)
print(f'Shape after: {data_08.shape[0]}')

Shape before: 121
Shape after: 102


In [12]:
# dropping the duplicates and nans
print(f'Shape before: {data_18.shape[0]}')
data_18 = cleanup_rows(df=data_18)
print(f'Shape after: {data_18.shape[0]}')

Shape before: 66
Shape after: 48


In [13]:
# top tweets
_tops = 10

In [14]:
# convert the imgUrl list to a string
data_08.imgUrl = data_08.imgUrl.apply(''.join)
data_18.imgUrl = data_18.imgUrl.apply(''.join)
data_08.head(1)

,tweetId,tweetUrl,timestamp,text,hashtags,hasMedia,imgUrl,likes,retweets,replies,isReplied
0,1088339520,/jimmyfallon/status/1088339520,2008-12-31 17:09:25,People on the streets already. Getting their s...,[],False,[],3,0,0,False


In [15]:
_top_tweets_08 = data_08.sort_values(by='likes', ascending=False).head(_tops)
_lowest_tweets_08 = data_08.sort_values(by='likes', ascending=False).tail(_tops)
_top_tweets_08.head(1)

,tweetId,tweetUrl,timestamp,text,hashtags,hasMedia,imgUrl,likes,retweets,replies,isReplied
120,1038620074,/jimmyfallon/status/1038620074,2008-12-04 18:34:54,I'm getting started on twitter....,[],False,[],449,201,33,True


In [16]:
_top_tweets_18 = data_18.sort_values(by='likes', ascending=False).head(_tops)
_lowest_tweets_18 = data_18.sort_values(by='likes', ascending=False).tail(_tops)
_top_tweets_18.head(1)

,tweetId,tweetUrl,timestamp,text,hashtags,hasMedia,imgUrl,likes,retweets,replies,isReplied
0,1079510916318679040,/jimmyfallon/status/1079510916318679040,2018-12-30 22:53:53,Oooohhhhhhhh my man #JIMIN (@BTS_twt) with a s...,['JIMIN'],False,[],486327,161313,6928,True


In [17]:
# list(_top_tweets.imgUrl)[0][0]

In [18]:
from IPython.display import HTML
from PIL import Image

In [19]:
HTML(_top_tweets_08[['text', 'likes', 'imgUrl']].to_html(formatters={'imgUrl': display_img}, escape=False))

,text,likes,imgUrl
120,I'm getting started on twitter....,449,
39,mac daddy'll make ya,102,
119,is listening to music :-),90,
108,Can I follow people from my iPhone?,61,
65,Early is the new lame.,21,
67,Thanks for the good wishes. The intelligent live on Twitter.,18,
40,happy christmas eve eve,17,
46,Went skiing today. First time ever. http://twitpic.com/v0su,15,
7,Just cheers'd my wife with hot sake. Hot or cold?,14,
16,Tis the night before Christmas,13,


In [20]:
HTML(_lowest_tweets_18[['text', 'likes', 'imgUrl']].to_html(formatters={'imgUrl': display_img}, escape=False))

,text,likes,imgUrl
44,Name That Song Challenge with @MileyCyrus https://youtu.be/OqkPildxhFw #FallonTonight,1098,
56,Trident Throwing with Jason Momoa! https://youtu.be/p5xIiIelscc #FallonTonight,1064,
64,"Fun show tonight: @MargotRobbie is here, @ElsieKFisher stops by, and music from @HootieTweets! #FallonTonight",1054,
61,"It’s Hashtags time! Tweet out a funny or weird time you had a holiday decoration disaster, and tag it with #DecorationDisaster. Could be on the show!",1037,
7,Tonight!! Willem Dafoe stops by and @HaileeSteinfeld is here to talk and perform! #FallonTonight,909,
62,"TONIGHT! Saoirse Ronan, @russwest44, and a performance from @alessiacara! #FallonTonight",904,
55,"TONIGHT! @icecube, @realamberheard, and @ElvisDuran! Plus, @icecube performs! Happy #ThankYouNoteFriday! #FallonTonight",869,
48,Can You Feel It? with @JLo and @MiloVentimiglia https://youtu.be/i33S77yegKo #FallonTonight,812,
46,Wooooowww. Well done. Never saw that one coming. #Jimberflake haaa.https://twitter.com/jkjohal64/status/1073272416242204678 …,780,
63,Loaded Questions with @MargotRobbie! https://youtu.be/zNjO6arjuAc #FallonTonight,663,


In [21]:
print('Characteristics of top tweets for 2008')
top_characteristics(_top_tweets_08, _tops)

Characteristics of top tweets for 2008


,hasMedia,hasHashtag,avarage hashtags,avarage text length,avarage likes,avarage retweet,avarage replies,avarage isReplied
10 Tweets,0,10,2.0,35.7,80.0,40.2,6.1,0.8


In [22]:
print('Characteristics of lowest tweets for 2008')
top_characteristics(_lowest_tweets_08, _tops)

Characteristics of lowest tweets for 2008


,hasMedia,hasHashtag,avarage hashtags,avarage text length,avarage likes,avarage retweet,avarage replies,avarage isReplied
10 Tweets,0,10,2.0,40.1,0.2,0.1,0.0,0.0


In [23]:
print('Characteristics of top tweets for 2018')
top_characteristics(_top_tweets_18, _tops)

Characteristics of top tweets for 2018


,hasMedia,hasHashtag,avarage hashtags,avarage text length,avarage likes,avarage retweet,avarage replies,avarage isReplied
10 Tweets,1,10,18.6,161.0,55625.3,16731.2,1153.5,1.0


In [24]:
print('Characteristics of lowest tweets for 2018')
top_characteristics(_lowest_tweets_18, _tops)

Characteristics of lowest tweets for 2018


,hasMedia,hasHashtag,avarage hashtags,avarage text length,avarage likes,avarage retweet,avarage replies,avarage isReplied
10 Tweets,0,10,19.5,102.4,919.0,89.3,72.2,1.0
